## import libraries

In [6]:
import sympy as sym
from sympy.abc import t

## main set up of constants and functions

In [10]:
#constants
dim1 = 10 #dim 1 of box
dim2 = dim1 #dim 2 of box (make it square)
lj = 2 #length of jack
mj = 1 #mass of the bulb at each end of jack
mb = 25 #give the walls of the box some large mass relative to jack
g = 9.8



In [12]:

#set up configuration variables for jack and box
#these will be used for deriving KE and PE equations for each component

#config variabls for jack
xj = sym.Function(r'x_j')(t)
yj = sym.Function(r'y_j')(t)
thj = sym.Function(r'\theta_j')(t)
#config vriables for box
xb = sym.Function(r'x_b')(t)
yb = sym.Function(r'y_b')(t)
thb = sym.Function(r'\theta_b')(t)

#consolidate config variables into array q
q = sym.Matrix([xj, yj, thj, xb, yb, thb])
qdot = q.diff(t)
qddot = qdot.diff(t)



## functions to handle frame transformations

In [28]:
#rotate a 2x2 matrix
def rot(th):
    return sym.Matrix([[sym.cos(th), -sym.sin(th)], [sym.sin(th), sym.cos(th)]])

#generate g transformation matrix (R is rotation component, t is translation component)
def transform_frame(rot, trans):
    g = sym.Matrix([
        [rot[0], rot[1], 0, trans[0]],
        [rot[2], rot[3], 0, trans[1]],
        [0, 0, 1, trans[2]],
        [0, 0, 0, 1]
    ])
    return g


#compute inverse of transformation matrix
def inverse_mat(mat):
    
    #extract rotation elements from input transform matrix mat
    R = sym.Matrix([
        [mat[0,0], mat[0,1], mat[0,2]],
        [mat[1,0], mat[1,1], mat[1,2]],
        [mat[2,0], mat[2,1], mat[2,2]]
    ])
    
    #we know it's a square matrix
    R_inv = R.T 
    
    #extract translational elements from input transform matrix mat
    p = sym.Matrix([
        mat[0,3],
        mat[1,3],
        mat[2,3]
    ])
    
    p_inv = -R_inv*p
    
    #now compute inverse matrix
    output = sym.Matrix([
        [R_inv[0,0], R_inv[0,1], R_inv[0,2], p_inv[0]],
        [R_inv[1,0], R_inv[1,1], R_inv[1,2], p_inv[1]],
        [R_inv[2,0], R_inv[2,1], R_inv[2,2], p_inv[2]],
        [0, 0, 0, 1]
    ])
    return output


## set up transformation matrices g_xx

In [33]:
#frame transformations

#world to jack
gwj = transform_frame(rot(q[2]), sym.Matrix([q[0], q[1], 0])) #rotation from world to jack frame
gjb = transform_frame(rot(0), sym.Matrix([0, -lj/2, 0]))
gjt = transform_frame(rot(0), sym.Matrix([0, lj/2, 0]))
gjl = transform_frame(rot(0), sym.Matrix([-lj/2, 0, 0]))
gjr = transform_frame(rot(0), sym.Matrix([lj/2, 0, 0]))

#jack to box
gwj_b = transform_frame(rot(q[5]), sym.Matrix([q[3], q[4], 0])) #rotation from jack to box frame
gjb_b = transform_frame(rot(0), sym.Matrix([0, -dim2/2, 0]))
gjb_t = transform_frame(rot(0), sym.Matrix([0, dim2/2, 0]))
gjb_l = transform_frame(rot(0), sym.Matrix([-dim1/2, 0, 0]))
gjb_r = transform_frame(rot(0), sym.Matrix([dim1/2, 0, 0]))

#transform from world frame to end points of jack 
gwb = gwj*gjb
gwt = gwj*gjt
gwl = gwj*gjl
gwr = gwj*gjr

#transform from jack center to wall points of box
gwb_b = gwj_b*gjb_b
gwb_t = gwj_b*gjb_t
gwb_l = gwj_b*gjb_l
gwb_r = gwj_b*gjb_r

#now transform from jack end points to center of box
Gjj_b = sym.simplify(inverse_mat(gwj)*gwj_b)
Gjb_b = sym.simplify(inverse_mat(gwb)*gwj_b)
Gjt_b = sym.simplify(inverse_mat(gwt)*gwj_b)
Gjl_b = sym.simplify(inverse_mat(gwl)*gwj_b)
Gjr_b = sym.simplify(inverse_mat(gwr)*gwj_b)



In [37]:
#transform world-to-jack endpoint frames to each box walls
#jack bottom end
Gbb_b = sym.simplify(inverse_mat(gwb)*gwb_b) #world to jack bottom pt, to jack bottom pt to box bottom wall
Gbt_b = sym.simplify(inverse_mat(gwb)*gwb_t) #world to jack bottom pt, to jack bottom pt to box top wall
Gbl_b = sym.simplify(inverse_mat(gwb)*gwb_l) #world to jack bottom pt, to jack bottom pt to box left wall
Gbr_b = sym.simplify(inverse_mat(gwb)*gwb_r) #world to jack bottom pt, to jack bottom pt to box right wall

#jack top end
Gtb_b = sym.simplify(inverse_mat(gwt)*gwb_b) #world to jack top pt, to jack bottom pt to box bottom wall
Gtt_b = sym.simplify(inverse_mat(gwt)*gwb_t) #world to jack top pt, to jack bottom pt to box top wall
Gtl_b = sym.simplify(inverse_mat(gwt)*gwb_l) #world to jack top pt, to jack bottom pt to box left wall
Gtr_b = sym.simplify(inverse_mat(gwt)*gwb_r) #world to jack top pt, to jack bottom pt to box right wall

#jack left end
Glb_b = sym.simplify(inverse_mat(gwl)*gwb_b) #world to jack left pt, to jack bottom pt to box bottom wall
Glt_b = sym.simplify(inverse_mat(gwl)*gwb_t) #world to jack left pt, to jack bottom pt to top bottom wall
Gll_b = sym.simplify(inverse_mat(gwl)*gwb_l) #world to jack left pt, to jack bottom pt to left bottom wall
Glr_b = sym.simplify(inverse_mat(gwl)*gwb_r) #world to jack left pt, to jack bottom pt to right bottom wall

#jack right end
Grb_b = sym.simplify(inverse_mat(gwr)*gwb_b) #world to jack right pt, to jack bottom pt to box bottom wall
Grt_b = sym.simplify(inverse_mat(gwr)*gwb_t) #world to jack right pt, to jack bottom pt to top bottom wall
Grl_b = sym.simplify(inverse_mat(gwr)*gwb_l) #world to jack right pt, to jack bottom pt to left bottom wall
Grr_b = sym.simplify(inverse_mat(gwr)*gwb_r) #world to jack right pt, to jack bottom pt to right bottom wall


## compute moment of inertia and body velocities to set up for KE and PE

In [48]:
from math import sqrt

def unhat(mat):
    return sym.Matrix([
        mat[0,3], mat[1,3], mat[2,3], mat[2,1], -mat[2,0], mat[1,0]
    ])

#Mass config
mb_tot = 4*mb
mj_tot = 4*mj
mb_dist = sqrt(2)*dim1/2
mj_dist = sqrt(2)*dim2/2
Jb = mb_tot*(dim1/2)**2
Jj = mj_tot*(dim2/2)**2

#Mass-Inertia
Ij = sym.Matrix([
        [mj_tot,0.,0.,0.,0.,0.],
        [0.,mj_tot,0.,0.,0.,0.],
        [0.,0.,mj_tot,0.,0.,0.],
        [0.,0.,0.,0.,0.,0.],
        [0.,0.,0.,0.,0.,0.],
        [0.,0.,0.,0.,0.,Jj]
    ])

Ij2 = sym.Matrix([
        [0.,0.,0.],
        [0.,0.,0.],
        [0.,0.,Jj]
    ])

Ib = sym.Matrix([
        [mb_tot,0.,0.,0.,0.,0.],
        [0.,mb_tot,0.,0.,0.,0.],
        [0.,0.,mb_tot,0.,0.,0.],
        [0.,0.,0.,0.,0.,0.],
        [0.,0.,0.,0.,0.,0.],
        [0.,0.,0.,0.,0.,Jb]
    ])

Ib2 = sym.Matrix([
        [0.,0.,0.],
        [0.,0.,0.],
        [0.,0.,Jb]
    ])

#body velocity
Vj = inverse_mat(gwj)*gwj.diff(t)
Vj = unhat(Vj)

In [51]:
display(sym.simplify(Vj))

Matrix([
[ sin(\theta_j(t))*Derivative(y_j(t), t) + cos(\theta_j(t))*Derivative(x_j(t), t)],
[-sin(\theta_j(t))*Derivative(x_j(t), t) + cos(\theta_j(t))*Derivative(y_j(t), t)],
[                                                                               0],
[                                                                               0],
[                                                                               0],
[                                                      Derivative(\theta_j(t), t)]])